# Metadata view를 통한 결론
* 탐색한 파일
    * <code>ptbxl_database.csv</code>
    * <code>scp_statements.csv</code>
* * *
* <code>ptbxl_database.csv</code>의 <code>scp_codes</code>의 문자열(딕셔너리)을 <code>json.loads()</code>를 통해 가져올 것

In [17]:
import pandas as pd
import json

In [7]:
ptbxl_db = pd.read_csv(r"data\PTB-XL\ptbxl_database.csv")

In [8]:
scp = pd.read_csv(r"data\PTB-XL\scp_statements.csv")

In [9]:
ptbxl_db.columns

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [10]:
scp.columns

Index(['Unnamed: 0', 'description', 'diagnostic', 'form', 'rhythm',
       'diagnostic_class', 'diagnostic_subclass', 'Statement Category',
       'SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code',
       'DICOM Code'],
      dtype='object')

In [32]:
print(ptbxl_db['scp_codes'][0])

{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}


In [29]:
print(ptbxl_db['scp_codes'][0].replace("\'", "\""))

{"NORM": 100.0, "LVOLT": 0.0, "SR": 0.0}


In [33]:
# 중요! : 이렇게 바꿔야 json 모듈에서 decode한다.

a = json.loads(ptbxl_db['scp_codes'][0].replace("\'", "\""))

In [34]:
type(a)

dict

In [36]:
for k, v in a.items():
    print(f"{k}: {v}")

NORM: 100.0
LVOLT: 0.0
SR: 0.0


In [11]:
scp.head()

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
2,DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
3,LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
4,NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7
